In [1]:
import json
from tqdm import tqdm_notebook
import matplotlib.pyplot as plt
import numpy as np
import inflect
import time
import gensim
engine = inflect.engine()

In [2]:
recipes = json.load(open('../generated/all_recipes_repr.json'))

In [3]:
len(recipes)

963396

In [4]:
def singularize(word):
    ingr = engine.singular_noun(word)
    return word if (not ingr) else ingr

def clean_ing_word(word) : return singularize(word).lower()

def clean_whole_ing(ing) : return " ".join([clean_ing_word(word) for word in ing.split(" ")])

Word2Vec training

In [5]:
start_time = time.time()
model = gensim.models.Word2Vec(recipes, min_count=2)

time_after_creation = time.time()
print("creation of model :", time_after_creation - start_time, "sec.")

model.train(recipes, total_examples=len(recipes), epochs=25)

time_after_training = time.time()
print("model training :", time_after_training - time_after_creation, "sec.")

model.save('../generated/food_embeddings.bin')

time_after_saving = time.time()
print("model saving :", time_after_saving - time_after_training, "sec.")

creation of model : 12.049983024597168 sec.
model training : 70.84084987640381 sec.
model saving : 0.01447439193725586 sec.


Results investigation

In [6]:
model.wv.most_similar("lamb")

/home/luke/anaconda3/lib/python3.6/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('lean lamb', 0.8593930006027222),
 ('lamb shoulder', 0.7749097347259521),
 ('lamb chop', 0.5847675204277039),
 ('eggplant', 0.5819183588027954),
 ('lamb shank', 0.5766223669052124),
 ('loin lamb', 0.5640079975128174),
 ('lamb rib chop', 0.5530146360397339),
 ('mutton', 0.5489798784255981),
 ('shoulder lamb chop', 0.538914680480957),
 ('chickpea', 0.5379897356033325)]

We see that the adjectives are creating new ingredients. Given we are only interested in flavours associations, we should ignore them. Indeed, shredded mushrooms and dried mushrooms should have the same flavour. 

In [7]:
model.wv.similarity(w1="salt", w2="pepper")

-0.4853339

In [8]:
 model.most_similar(positive=['rice', 'tomato'], negative=['pasta'], topn=1)

/home/luke/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('beef tongue', 0.5083563923835754)]